## Plotting of profile results

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import gridspec

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database, hyswan_db


In [2]:
from teslakit.database import SplitStorage
def Load_SIM_NEARSHORE_all(db, vns=[], decode_times=False, use_cftime=False, prf=[]):
        ps = db.paths.site.SIMULATION.nearshore

        # locate simulations
        #sims = sorted([x for x in os.listdir(ps) if x.isdigit() and op.isdir(op.join(ps, x))])
        sims = sorted([x for x in os.listdir(ps) if x.endswith("_prf_"+str(prf))])

        # read all simulations
        for c, nm in enumerate(sims):
            ps_sim = op.join(ps, nm)
            s = SplitStorage(ps_sim)

            # read time and variables from first file
            if c==0:
                ds = s.Load(vns=vns, decode_times=False)

                # generate output xarray.Dataset, dims and vars
                out = xr.Dataset({}, coords={'time': ds.time, 'n_sim':range(len(sims))})
                for vds in ds.variables:
                    if vds == 'time': continue
                    out[vds] = (('time', 'n_sim',), np.nan*np.zeros((len(out.time), len(out.n_sim))))
                    out[vds].loc[dict(n_sim=c)] = ds[vds]

            else:
                ds = s.Load(vns=vns, decode_times=False)

                # fill output xarray.Dataset
                for vds in ds.variables:
                    if vds == 'time': continue
                    out[vds].loc[dict(n_sim=c)] = ds[vds]

        # optional decode times to cftime 
        if decode_times:
            out = xr.decode_cf(out, use_cftime=use_cftime)

        return out

In [3]:
# --------------------------------------
# Teslakit database
p_data = r'/media/administrador/HD/Dropbox/Guam/teslakit/data'
# p_data=r'/Users/laurac/Dropbox/Guam/teslakit/data'
# offshore
db = Database(p_data)
db.SetSite('GUAM')

# climate change - S6
db_S6 = Database(p_data)
db_S6.SetSite('GUAM_CC_S6')

# hyswan simulation database
db_sim = hyswan_db(db.paths.site.HYSWAN.sim)
db_sim_S6 = hyswan_db(db_S6.paths.site.HYSWAN.sim)

### Set profile and load data

In [4]:
def Plot_profile(profile):
    colors=['royalblue','crimson','gold','darkmagenta','darkgreen','darkorange','mediumpurple','coral','pink','lightgreen','darkgreen','darkorange']
    fig=plt.figure(figsize=[17,4])
    gs1=gridspec.GridSpec(1,1)
    ax=fig.add_subplot(gs1[0])

    ax.plot(profile.Distance_profile, profile.Elevation,linewidth=3,color=colors[prf],alpha=0.7,label='Profile: ' + str(prf))
    s=np.where(profile.Elevation<0)[0][0]
    ax.plot(profile.Distance_profile[s],profile.Elevation[s],'s',color=colors[prf],markersize=10)
    ax.plot([0,1500],[0,0],':',color='plum',alpha=0.7)
    ax.plot([0,1500],[np.nanmin(profile.Elevation),np.nanmin(profile.Elevation)],':',color='plum',alpha=0.7)

    ax.set_xlabel(r'Distance (m)', fontsize=14)
    ax.set_ylabel(r'Elevation (m)', fontsize=14)
    ax.legend()
    ax.set_xlim([0,np.nanmax(profile.Distance_profile)])

In [6]:
# prf=11

for prf in range(12):
    
    print('Profile_ ' + str(prf))
    
    profiles=xr.open_dataset('/media/administrador/HD/Dropbox/Guam/bati guam/Profiles_Guam_curt.nc')
    profile=profiles.sel(profile=prf)
    profile
#     Plot_profile(profile)
    
    # Simulation
    sim=Load_SIM_NEARSHORE_all(db_S6,vns=['Hs','Tp','Dir'], decode_times=False, use_cftime=False, prf=prf)
    sim2=db_S6.Load_SIM_OFFSHORE_all(vns=['level','wind_dir','wind_speed'], decode_times=False, use_cftime=False) #Level=SS+AT+MMSL
    sim['level']=sim2.level
    sim['wind_dir']=sim2.wind_dir
    sim['wind_speed']=sim2.wind_speed
    
    SIM=sim.to_dataframe().reset_index()
    SIM.to_pickle(os.path.join(db_S6.paths.site.SIMULATION.nearshore,'Simulations_profile_'+str(prf)))



Profile_ 0
Profile_ 1
Profile_ 2
Profile_ 3
Profile_ 4
Profile_ 5
Profile_ 6
Profile_ 7
Profile_ 8
Profile_ 9
Profile_ 10
Profile_ 11
